In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import os
import psycopg2

In [2]:
def get_data(path):
    try:
        data = pd.read_csv(path)
        df = pd.DataFrame(data)
    except Exception as e:
        print(e)
    return df

In [3]:
#data frames original and copy date 10.03
listings_10_03_summary = get_data('../airbnb_data_vienna/10_03/summary_visualization/listings.csv')
listings_10_03 = get_data('../airbnb_data_vienna/10_03/listings.csv')
listings_10_03['id'].count()

11452

In [4]:
#data frames original and copy date 07.06
listings_11_09_summary = get_data('../airbnb_data_vienna/11_09/summary_visualization/listings.csv')
listings_11_09 = get_data('../airbnb_data_vienna/11_09/listings.csv')
listings_11_09_summary['id'].count()

11797

In [5]:
#data frames original and copy date 11.09
listings_07_06_summary = get_data('../airbnb_data_vienna/07_06/summary_visualization/listings.csv')
listings_07_06 = get_data('../airbnb_data_vienna/07_06/listings.csv')
listings_07_06_summary['id'].count()

11274

In [6]:
# Merge summary 10_03 with detailed listing for host_response_time, host_response_rate, host_accceptance_rate,
# host_ist_superhost
listings_10_03_columns = ['id', 'host_response_time', 'host_url', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'property_type', 'accommodates', 
'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
'review_scores_value', 'instant_bookable']
listings_10_03_merged_org = listings_10_03_summary.merge(listings_10_03[listings_10_03_columns], on='id', how='left')
listings_10_03_merged = listings_10_03_merged_org.copy()

In [7]:
# Merge summary 07_06 with detailed listing for host_response_time, host_response_rate, host_accceptance_rate,
# host_ist_superhost...
listings_07_06_columns = ['id', 'host_response_time', 'host_url', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'property_type', 'accommodates', 
'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
'review_scores_value', 'instant_bookable']
listings_07_06_merged_org = listings_07_06_summary.merge(listings_07_06[listings_07_06_columns], on='id', how='left')
listings_07_06_merged = listings_07_06_merged_org.copy()
listings_07_06_merged.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license',
       'host_response_time', 'host_url', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'property_type',
       'accommodates', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable'],
      dtype='object')

In [8]:
# Merge summary 11_09 with detailed listing for host_response_time, host_response_rate, host_accceptance_rate,
# host_ist_superhost...
listings_11_09_columns = ['id', 'host_response_time', 'host_url', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'property_type', 'accommodates', 
'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
'review_scores_value', 'instant_bookable']
listings_11_09_merged_org = listings_11_09_summary.merge(listings_11_09[listings_11_09_columns], on='id', how='left')
listings_11_09_merged = listings_11_09_merged_org.copy()
listings_11_09_merged['id'].count()

11797

In [9]:
#merge all dataframes into one listing, drop duplicates
data_frames_to_merge = [listings_10_03_merged, listings_07_06_merged, listings_11_09_merged]
df_listings_merged = reduce(lambda left,right: pd.merge(left,right, on=['id'], how='outer', suffixes=('', '_DROP')), data_frames_to_merge).filter(regex='^(?!.*_DROP)')
df_listings_merged_copy = df_listings_merged.copy()
df_listings_merged_copy = df_listings_merged_copy.drop(columns=['neighbourhood_group', 'license'])
df_listings_merged_copy.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm', 'host_response_time', 'host_url',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'property_type', 'accommodates', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable'],
      dtype='object')

In [10]:
#clean table names, lower case letters, remove all white spaces - optional
file = 'list ings$'
clean_tbl_name = file.lower().replace(" ", "_").replace("?", "") \
    .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "") \
        .replace(")", "").replace(r"(", "").replace("$", "")
clean_tbl_name

'list_ings'

In [11]:
#clean header names, lower case letters, remove all white spaces
df_listings_merged_copy.columns = [x.lower().replace(" ", "_").replace("?", "") \
    .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "") \
        .replace(")", "").replace(r"(", "").replace("$", "") for x in df_listings_merged_copy.columns]
df_listings_merged_copy.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm', 'host_response_time', 'host_url',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'property_type', 'accommodates', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable'],
      dtype='object')

In [12]:
#count null values and drop rows with multiple null values
df_listings_merged_copy['null count'] = df_listings_merged_copy.isnull().sum(axis=1)
mask = (df_listings_merged_copy['null count'] >= 15)
df_listings_merged_copy.drop(df_listings_merged_copy[mask].index, inplace=True)
print(df_listings_merged_copy['number_of_reviews_ltm'].isnull().any())
print(df_listings_merged_copy.isnull().sum(axis=1))

False
0         0
1         3
2         3
3        12
4         3
         ..
11447     0
11448     0
11449     0
11450     0
11451     9
Length: 11452, dtype: int64


In [13]:
#fill null values reviews_per_month
df_listings_merged_na = df_listings_merged_copy.copy()
df_listings_merged_na['reviews_per_month'] = df_listings_merged_na['reviews_per_month'].fillna(0)
print(df_listings_merged_na['availability_365'].isnull().any())

False


In [14]:
# check object columns for null values
df_listings_merged_na['host_acceptance_rate'].unique() 

array(['100%', nan, '99%', '79%', '74%', '93%', '30%', '92%', '0%', '59%',
       '98%', '96%', '50%', '95%', '83%', '40%', '88%', '97%', '94%',
       '72%', '39%', '84%', '91%', '28%', '78%', '75%', '20%', '87%',
       '37%', '25%', '44%', '57%', '33%', '21%', '82%', '56%', '63%',
       '80%', '85%', '46%', '69%', '86%', '67%', '71%', '64%', '45%',
       '90%', '65%', '54%', '38%', '31%', '29%', '18%', '61%', '89%',
       '77%', '70%', '55%', '6%', '47%', '36%', '51%', '17%', '81%',
       '76%', '60%', '58%', '22%', '49%', '66%', '27%', '42%', '52%',
       '53%', '73%', '15%', '14%', '10%', '11%', '43%', '62%', '13%',
       '48%', '24%', '26%', '41%', '68%', '12%', '35%', '4%', '19%', '8%'],
      dtype=object)

In [15]:
# fill 0% for rates
df_listings_merged_na = df_listings_merged_na.fillna({'host_response_rate': '0%', 'host_acceptance_rate': '0%'})

In [16]:
#fill No response for host_response_time
df_listings_merged_na['host_response_time'] = df_listings_merged_na['host_response_time'].fillna('no response')

In [17]:
# Fill Unknown for name host_name
df_listings_merged_na = df_listings_merged_na.fillna({'name': 'unknown', 'host_name': 'unknown'})

In [18]:
#check objects for null values
df_listings_merged_na.loc[::,'property_type':]

,property_type,accommodates,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,null count
0,Entire rental unit,2.0,4.74,4.96,4.87,4.78,4.91,4.78,4.91,t,0
1,Private room in rental unit,1.0,5.00,5.00,5.00,5.00,5.00,5.00,5.00,t,3
2,Shared room in rental unit,2.0,5.00,5.00,5.00,5.00,5.00,5.00,5.00,t,3
3,Private room in rental unit,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,12
4,Entire rental unit,2.0,4.78,4.78,4.89,4.78,5.00,4.67,4.89,f,3
...,...,...,...,...,...,...,...,...,...,...,...
11447,Private room in rental unit,2.0,5.00,5.00,5.00,5.00,5.00,5.00,5.00,t,0
11448,Private room in bed and breakfast,16.0,4.00,5.00,5.00,5.00,5.00,4.00,4.00,f,0
11449,Entire condominium (condo),2.0,5.00,5.00,4.83,5.00,5.00,5.00,5.00,t,0
11450,Entire rental unit,4.0,4.85,4.85,4.92,4.93,4.92,4.75,4.87,f,0


In [19]:
# last_review transform to date, null values stay
df_listings_merged_na['last_review'] = pd.to_datetime(df_listings_merged_na['last_review'])

In [20]:
# host_is_superhost/instant bookable transform to boolean 1 and 0, null values stay
df_listings_merged_na['host_is_superhost'] = df_listings_merged_na['host_is_superhost'].replace({'t': '1', 'f':'0'})
df_listings_merged_na['host_is_superhost'] = pd.to_numeric(df_listings_merged_na['host_is_superhost'], errors='coerce')
df_listings_merged_na['instant_bookable'] = df_listings_merged_na['instant_bookable'].replace({'t': '1', 'f':'0'})
df_listings_merged_na['instant_bookable'] = pd.to_numeric(df_listings_merged_na['instant_bookable'], errors='coerce')

In [21]:
# transform percentage rates from object to float
df_listings_merged_na['host_response_rate'] = df_listings_merged_na['host_response_rate'].str.rstrip('%').astype('float') / 100.0
df_listings_merged_na['host_acceptance_rate'] = df_listings_merged_na['host_acceptance_rate'].str.rstrip('%').astype('float') / 100.0

In [22]:
# transform id to float, accommodates to int
df_listings_merged_na = df_listings_merged_na.astype({'id' : float})
df_listings_merged_na = df_listings_merged_na.astype({'accommodates' : 'int64'})

In [23]:
#rating averages
df_listings_merged_na['avg_score_rating'] = df_listings_merged_na.loc[::, 'review_scores_rating':'review_scores_value'].mean(numeric_only=True, skipna=True, axis=1).round(2)
df_listings_merged_na = df_listings_merged_na.drop(columns='null count')

In [24]:
df_listings_merged_na.head()
df_listings_merged_na['number_of_reviews'].mode()
df_listings_merged_na.describe().T

,count,mean,std,min,25%,50%,75%,max
id,11452.0,1.694190e+16,9.587092e+16,15883.00000,1.636199e+07,3.096924e+07,4.402723e+07,5.781258e+17
host_id,11452.0,1.226891e+08,1.287431e+08,19469.00000,1.593529e+07,6.624488e+07,2.109623e+08,4.506911e+08
latitude,11452.0,4.820515e+01,2.043823e-02,48.12528,4.819156e+01,4.820435e+01,4.821853e+01,4.832007e+01
longitude,11452.0,1.636090e+01,3.531200e-02,16.19317,1.633910e+01,1.635807e+01,1.638088e+01,1.654662e+01
price,11452.0,8.251877e+01,2.111196e+02,0.00000,4.000000e+01,6.000000e+01,8.900000e+01,9.999000e+03
minimum_nights,11452.0,6.835400e+00,2.804493e+01,1.00000,1.000000e+00,2.000000e+00,3.000000e+00,1.125000e+03
number_of_reviews,11452.0,3.259562e+01,6.283881e+01,0.00000,1.000000e+00,7.000000e+00,3.400000e+01,6.450000e+02
reviews_per_month,11452.0,8.681104e-01,1.407903e+00,0.00000,4.000000e-02,2.900000e-01,1.160000e+00,4.714000e+01
calculated_host_listings_count,11452.0,1.127559e+01,2.105029e+01,1.00000,1.000000e+00,2.000000e+00,8.000000e+00,1.010000e+02
availability_365,11452.0,1.442295e+02,1.386030e+02,0.00000,0.000000e+00,1.015000e+02,2.860000e+02,3.650000e+02


In [82]:
# create table listings
# (
#     id                                 int,
#     name                               varchar,
#     host_id                            float,
#     host_name                          varchar,
#     neighbourhood                      varchar,
#     latitude                          float,
#     longitude                         float,
#     room_type                          varchar,
#     price                             float,
#     minimum_nights                    float,
#     number_of_reviews                 float,
#     last_review                        timestamp,
#     reviews_per_month                 float,
#     calculated_host_listings_count    float,
#     availability_365                  float,
#     number_of_reviews_ltm             float,
#     host_response_time                 varchar,
#     host_response_rate                 float,
#     host_acceptance_rate               float,
#     host_is_superhost                  float
# );

In [25]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'int',
    'datetime64[ns]' : 'timestamp'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64[ns]': 'timestamp'}

In [26]:
#columns for postgres database
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df_listings_merged_na.columns, df_listings_merged_na.dtypes.replace(replacements)))
col_str

'id float, name varchar, host_id float, host_name varchar, neighbourhood varchar, latitude float, longitude float, room_type varchar, price float, minimum_nights float, number_of_reviews float, last_review timestamp, reviews_per_month float, calculated_host_listings_count float, availability_365 float, number_of_reviews_ltm float, host_response_time varchar, host_url varchar, host_response_rate float, host_acceptance_rate float, host_is_superhost float, property_type varchar, accommodates int, review_scores_rating float, review_scores_accuracy float, review_scores_cleanliness float, review_scores_checkin float, review_scores_communication float, review_scores_location float, review_scores_value float, instant_bookable int, avg_score_rating float'

In [92]:
#Open a database connection
conn_string = "host=database-airbnb.cl4bob5uux9z.us-west-1.rds.amazonaws.com \
    dbname='postgres' \
        user='postgres' password='geslozapostgre123'"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('Opened database successfully')

Opened database successfully


In [93]:
#drop tables with the same name
cursor.execute("drop table if exists listings;")

In [94]:
#create table
cursor.execute("create table listings \
    (id float, name varchar, host_id float, host_name varchar, neighbourhood varchar, latitude float, longitude float, room_type varchar, \
        price float, minimum_nights float, number_of_reviews float, last_review timestamp, reviews_per_month float, \
            calculated_host_listings_count float, availability_365 float, number_of_reviews_ltm float, host_response_time varchar, \
                host_response_rate float, host_acceptance_rate float, host_is_superhost float, \
                    property_type varchar, accommodates int, review_scores_rating float, review_scores_accuracy float, \
                        review_scores_cleanliness float, review_scores_checkin float, review_scores_communication float, \
                            review_scores_location float, review_scores_value float, instant_bookable int, avg_score_rating float)")

In [27]:
# Save to csv
df_listings_merged_na.to_csv('../airbnb_data_vienna/csv/listings.csv', header=df_listings_merged_na.columns, index=False, encoding='utf-8')

In [28]:
#get listing link
df_host_url = pd.read_csv('../airbnb_data_vienna/csv/listings.csv')

In [36]:
filter = df_host_url.query("host_id == 757295.0 | host_id == 318961878.0")
filter['host_url'].unique()

array(['https://www.airbnb.com/users/show/757295',
       'https://www.airbnb.com/users/show/318961878'], dtype=object)

In [96]:
#open the csv file, save it as an object
listings_file = open('../airbnb_data_vienna/csv/listings.csv', encoding='utf-8')
print('File opened in memory')

File opened in memory


In [97]:
# upload to db
sql_query = """
COPY listings FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""
cursor.copy_expert(sql=sql_query, file= listings_file)
print('File copied to db')

File copied to db


In [98]:
cursor.execute("grant select on table listings to public")
conn.commit()
cursor.close()
print('Table listings immported to db completed')

Table listings immported to db completed
